In [2]:
# Let's start by retrieving some Magic card images using the Scryfall API
# https://scryfall.com/docs/api
# We'll be using the Scryfall python library
# https://github.com/NandaScott/Scrython

In [85]:
import requests
from requests.exceptions import TooManyRedirects, ReadTimeout, ConnectionError
import json
import pprint
import time
import shutil
import os
import os.path

URL='https://api.scryfall.com'
OUTDIR='/Users/agussman/Documents/DeepLens/deeplens-magiccards/data'


In [ ]:
r = requests.get('https://api.scryfall.com/cards/search?order=set&q=e:dom')

In [45]:
results = r.json()['data']
for result in results[:10]:
    #card = result
    #if 'all_parts' in result
    #print(result['image_uris']['png'])
    time.sleep(0.25)
    image_url = result['image_uris']['png']
    filename = "{}_{num:03d}.png".format(result['set'], num=int(result['collector_number']))
    filename = os.path.join(OUTDIR, result['set'], filename)
    print(filename)
    print("{} -> {}".format(image_url, filename))
    
    r = requests.get(image_url, stream=True)
    
    with open(filename, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=128):
            fd.write(chunk)
    
    '''
    if r.status_code == 200:
        with open(os.path.join(OUTDIR, result['set'], filename), 'wb') as out_file:
            shutil.copyfileobj(response.raw(), out_file)
    else:
        print("Nope. {}".format(response.status_code))
    '''
    
    
#pprint.pprint(results[0:2])

/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_001.png
https://img.scryfall.com/cards/png/en/dom/1.png?1524789999 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_001.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_002.png
https://img.scryfall.com/cards/png/en/dom/2.png?1525862131 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_002.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_003.png
https://img.scryfall.com/cards/png/en/dom/3.png?1525862133 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_003.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_004.png
https://img.scryfall.com/cards/png/en/dom/4.png?1524790027 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_004.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_005.png
https://img.scryfall.com/cards/png/en/dom/5.png?1528770903 -> /Users

In [73]:
# Next let's download some pictures of tables
table_list_url = 'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n03201035'
r = requests.get(table_list_url)

file_urls = r.text.splitlines()

In [86]:
for num, table_url in enumerate(file_urls[10:100]):
    print(num, table_url)
    time.sleep(0.25)
    filename = "{num:003d}_{original}".format(num=num, original=os.path.basename(table_url))
    filename = os.path.join(OUTDIR, 'tables', filename)
    print(filename)
    print("{} -> {}".format(table_url, filename))
    
    try:
        r = requests.get(table_url, stream=True, timeout=1)
        if r.status_code == 200:
            with open(filename, 'wb') as fd:
                for chunk in r.iter_content(chunk_size=128):
                    fd.write(chunk)
        else:
            print("SKIP: Problem downloading: {}".format(r.status_code))
    except TooManyRedirects:
        print("SKIP: TooManyRedirects")
    except ReadTimeout:
        print("SKIP: ReadTimeout")
    except ConnectionError:
        print("SKIP: ConnectionError")
    


0 http://farm3.static.flickr.com/2147/2346955347_6a505656d0.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/000_2346955347_6a505656d0.jpg
http://farm3.static.flickr.com/2147/2346955347_6a505656d0.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/000_2346955347_6a505656d0.jpg
1 http://furniturefromhome.com/images/store/554417/760x600/image.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/001_image.jpg
http://furniturefromhome.com/images/store/554417/760x600/image.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/001_image.jpg
2 http://farm3.static.flickr.com/2299/2058125616_b17f969864.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/002_2058125616_b17f969864.jpg
http://farm3.static.flickr.com/2299/2058125616_b17f969864.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/002_2058125616_b17f969864.jpg
3 http://farm3.static.flickr.com/2164/1670651931_26aa

26 http://farm1.static.flickr.com/47/141254418_2c89bcd242.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/026_141254418_2c89bcd242.jpg
http://farm1.static.flickr.com/47/141254418_2c89bcd242.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/026_141254418_2c89bcd242.jpg
27 http://www.gowfb.com/images/imagecache/Repertoire-Pedestal-Table-Upholstered-Chairs.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/027_Repertoire-Pedestal-Table-Upholstered-Chairs.jpg
http://www.gowfb.com/images/imagecache/Repertoire-Pedestal-Table-Upholstered-Chairs.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/027_Repertoire-Pedestal-Table-Upholstered-Chairs.jpg
SKIP: Problem downloading: 404
28 http://farm2.static.flickr.com/1082/1286228285_2ab6688eb8.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/028_1286228285_2ab6688eb8.jpg
http://farm2.static.flickr.com/1082/1286228285_2ab6688eb8.jpg -> /Use

SKIP: ConnectionError
50 http://farm3.static.flickr.com/2404/2461862083_aa1655221a.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/050_2461862083_aa1655221a.jpg
http://farm3.static.flickr.com/2404/2461862083_aa1655221a.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/050_2461862083_aa1655221a.jpg
51 http://farm4.static.flickr.com/3141/2574206280_779d6b918b.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/051_2574206280_779d6b918b.jpg
http://farm4.static.flickr.com/3141/2574206280_779d6b918b.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/051_2574206280_779d6b918b.jpg
52 http://farm1.static.flickr.com/187/410163434_47b1653bf4.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/052_410163434_47b1653bf4.jpg
http://farm1.static.flickr.com/187/410163434_47b1653bf4.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/052_410163434_47b1653bf4.jpg
53 http://fa

75 http://www.avetexfurniture.com/file.asp?ID=1133&table=Goods&number=2
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/075_file.asp?ID=1133&table=Goods&number=2
http://www.avetexfurniture.com/file.asp?ID=1133&table=Goods&number=2 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/075_file.asp?ID=1133&table=Goods&number=2
SKIP: ConnectionError
76 http://www.dapperdecor.com/store/p_furniture_7.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/076_p_furniture_7.jpg
http://www.dapperdecor.com/store/p_furniture_7.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/076_p_furniture_7.jpg
77 http://www.letmom.com/choose/furniture/images/efr/hill-gd-table-1.jpg
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/077_hill-gd-table-1.jpg
http://www.letmom.com/choose/furniture/images/efr/hill-gd-table-1.jpg -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/tables/077_hill-gd-table-1.jpg
SK

In [64]:
import os
for num, table_url in enumerate(r.text.splitlines()[:10]):
    print(num, table_url, os.path.basename(table_url))

0 http://www.american-furniture-warehouse.com/images/Furniture/stsk4872t.jpg stsk4872t.jpg
1 http://farm3.static.flickr.com/2094/2156394956_636a8e82b7.jpg 2156394956_636a8e82b7.jpg
2 http://farm1.static.flickr.com/119/255091104_0110d3fb49.jpg 255091104_0110d3fb49.jpg
3 http://farm1.static.flickr.com/183/424762396_89fdd67fae.jpg 424762396_89fdd67fae.jpg
4 http://farm4.static.flickr.com/3064/2701347106_5072432f52.jpg 2701347106_5072432f52.jpg
5 http://resaleauctions.com/furniture/diningtable.JPG diningtable.JPG
6 http://farm1.static.flickr.com/19/97833898_b311b6e9cc.jpg 97833898_b311b6e9cc.jpg
7 http://farm4.static.flickr.com/3065/2343394045_bd561e7ab1.jpg 2343394045_bd561e7ab1.jpg
8 http://farm2.static.flickr.com/1093/1423281989_ef0907f396.jpg 1423281989_ef0907f396.jpg
9 http://farm2.static.flickr.com/1085/1481438320_7df49bd11c.jpg 1481438320_7df49bd11c.jpg


In [4]:
import time

query = input("Type the name of the set: ")

time.sleep(0.05)
sets = scrython.sets.Sets()

for i in range(sets.data_length()):
    if sets.set_name(i) == query:
        print("Set code:", sets.set_code(i).upper())
        break
    else:
        continue

Type the name of the set: dom


RuntimeError: Cannot run the event loop while another loop is running